In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
# Random
import random
random.seed = 42
# Thread management (sleep purposes)
import time
# Operating System management
import os
import selenium
# Selenium specific libraries
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np

import pickle

In [2]:
if not os.path.exists("portales/pisos_com"):
  os.makedirs("portales/pisos_com")

In [40]:
configurations = {
    'url' : 'https://www.pisos.com/viviendas/valencia_capital_zona_urbana/'}
service = Service(executable_path='chromedriver.exe')

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
#options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [41]:
service = Service(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

driver.get(configurations["url"])
driver.maximize_window()
time.sleep(np.random.normal(2,0.3)) # segundos

driver.save_screenshot("portales/pisoscom/00-homepage.png")

False

In [27]:
driver.find_element(by=By.ID, value="didomi-notice-agree-button").click()
time.sleep(np.random.normal(2,0.3)) # segundos
driver.save_screenshot("portales/fotocasa/01-homepage-no-cookies.png")

False

In [28]:
#ESTE CÓDIGO NOS SIRVE SI ENTRAMOS AL BUSCADOR INICIALMENTE, PERO NOSOTROS ENTRAMOS AL MAPA PARA OBTENER LAS URL DE LOS DISTRITOS
# ----------------------------------------------------------

# campo_busqueda = driver.find_element(by=By.CLASS_NAME, value="w-autocomplete__input")
# time.sleep(np.random.normal(2,0.3)) # segundos
# consulta = "valencia poblacion"
# campo_busqueda.send_keys(consulta)
# time.sleep(np.random.normal(2))
# # Simular la presión de la tecla Enter para realizar la búsqueda (si es necesario)
# campo_busqueda.send_keys(Keys.RETURN)

# time.sleep(np.random.normal(2)) # segundos
# driver.save_screenshot("portales/indomio/02-homepage-no-cookies.png")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".w-autocomplete__input"}
  (Session info: chrome=124.0.6367.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF719AF7032+63090]
	(No symbol) [0x00007FF719A62C82]
	(No symbol) [0x00007FF7198FEC65]
	(No symbol) [0x00007FF71994499D]
	(No symbol) [0x00007FF719944ADC]
	(No symbol) [0x00007FF719985B37]
	(No symbol) [0x00007FF71996701F]
	(No symbol) [0x00007FF719983412]
	(No symbol) [0x00007FF719966D83]
	(No symbol) [0x00007FF7199383A8]
	(No symbol) [0x00007FF719939441]
	GetHandleVerifier [0x00007FF719EF25AD+4238317]
	GetHandleVerifier [0x00007FF719F2F70D+4488525]
	GetHandleVerifier [0x00007FF719F279EF+4456495]
	GetHandleVerifier [0x00007FF719BD0576+953270]
	(No symbol) [0x00007FF719A6E54F]
	(No symbol) [0x00007FF719A69224]
	(No symbol) [0x0000

In [8]:
enlaces1 = driver.find_elements(by=By.CLASS_NAME, value= 'item')
distritos={}
for enlace in enlaces1:
    nombre_distrito = enlace.get_attribute("text")
    url_distrito = enlace.get_attribute("href")
    if nombre_distrito != None or nombre_distrito != None:
        print("Distrito:", nombre_distrito)
        print("URL:", url_distrito)
        distritos[nombre_distrito] = url_distrito

Distrito: Algirós (56)
URL: https://www.pisos.com/venta/pisos-algiros/
Distrito: Benicalap (97)
URL: https://www.pisos.com/viviendas/valencia_capital_benicalap/
Distrito: Benimaclet (11)
URL: https://www.pisos.com/venta/pisos-benimaclet_distrito/
Distrito: Camins al Grau (125)
URL: https://www.pisos.com/viviendas/camins_al_grau/
Distrito: Campanar (94)
URL: https://www.pisos.com/viviendas/valencia_capital_campanar/
Distrito: Ciutat Vella (260)
URL: https://www.pisos.com/viviendas/valencia_capital_ciutat_vella/
Distrito: El Pla del Real (57)
URL: https://www.pisos.com/venta/pisos-el_pla_del_real/
Distrito: Extramurs (145)
URL: https://www.pisos.com/viviendas/extramurs/
Distrito: Jesús (88)
URL: https://www.pisos.com/viviendas/jesus/
Distrito: L'Eixample (191)
URL: https://www.pisos.com/viviendas/valencia_capital_eixample/
Distrito: L'Olivereta (102)
URL: https://www.pisos.com/viviendas/olivereta/
Distrito: La Saïdia (58)
URL: https://www.pisos.com/venta/pisos-la_saidia/
Distrito: Patrai

In [9]:
distritos = dict(list(distritos.items())[:19])

In [49]:
# Suponiendo que distritos es tu diccionario de distritos con nombre y URL

urls_barrios = {}

for nombre_distrito, url_distrito in distritos.items():
    driver.get(url_distrito)
    time.sleep(2)  # Espera un tiempo para cargar la página completamente
    
    # Encuentra todos los elementos con id que comienza con "MapMap"
    barrios = driver.find_elements(By.CSS_SELECTOR, '[id^="MapMap"]')
    
    # Comprueba si hay elementos de selección de barrios
    if barrios:
        if url_distrito.startswith("https://www.pisos.com/venta/"):
        # Si hay elementos de selección de barrios, extrae las URL de los barrios y guárdalas en el diccionario urls_barrios
            urls_barrios[nombre_distrito] = [barrio.get_attribute("href") for barrio in barrios]
    else:
        # Si no hay elementos de selección de barrios, asume que la página contiene anuncios directamente y guarda la URL del distrito
        break




{}


In [69]:
urls_barrios

{'Algirós (56)': ['https://www.pisos.com/venta/pisos-algiros/'],
 'Benicalap (97)': [None,
  'https://www.pisos.com/venta/pisos-benicalap_barrio/',
  'https://www.pisos.com/venta/pisos-ciutat_fallera/',
  'https://www.pisos.com/viviendas/valencia_capital_campanar/',
  'https://www.pisos.com/viviendas/horta_nord/',
  'https://www.pisos.com/viviendas/la_saidia/',
  'https://www.pisos.com/viviendas/pobles_de_oest/',
  'https://www.pisos.com/viviendas/pobles_del_nord/',
  'https://www.pisos.com/viviendas/rascanya/'],
 'Benimaclet (11)': ['https://www.pisos.com/venta/pisos-benimaclet_distrito/'],
 'Camins al Grau (125)': [None,
  'https://www.pisos.com/venta/pisos-aiora/',
  'https://www.pisos.com/venta/pisos-albors/',
  'https://www.pisos.com/venta/pisos-cami_fondo/',
  'https://www.pisos.com/venta/pisos-la_creu_del_grau/',
  'https://www.pisos.com/venta/pisos-penya_roja/',
  'https://www.pisos.com/viviendas/algiros/',
  'https://www.pisos.com/viviendas/el_pla_del_real/',
  'https://www.pi

In [72]:
urls_limpias = []
for distrito, lista_u in urls_barrios.items():
    for i in range(len(lista_u)):
        if str(lista_u[i]).startswith("https://www.pisos.com/venta/"):
            urls_limpias.append(lista_u[i])

In [74]:
with open('barrios_vlc_pisos_com.pkl', 'wb') as f:
    pickle.dump(urls_limpias, f)

In [6]:
with open('barrios_vlc_pisos_com.pkl', 'rb') as f:
    urls_barrios = pickle.load(f)

In [7]:
urls_barrios

['https://www.pisos.com/venta/pisos-algiros/',
 'https://www.pisos.com/venta/pisos-benicalap_barrio/',
 'https://www.pisos.com/venta/pisos-ciutat_fallera/',
 'https://www.pisos.com/venta/pisos-benimaclet_distrito/',
 'https://www.pisos.com/venta/pisos-aiora/',
 'https://www.pisos.com/venta/pisos-albors/',
 'https://www.pisos.com/venta/pisos-cami_fondo/',
 'https://www.pisos.com/venta/pisos-la_creu_del_grau/',
 'https://www.pisos.com/venta/pisos-penya_roja/',
 'https://www.pisos.com/venta/pisos-campanar_barrio/',
 'https://www.pisos.com/venta/pisos-el_calvari/',
 'https://www.pisos.com/venta/pisos-les_tendetes/',
 'https://www.pisos.com/venta/pisos-sant_pau/',
 'https://www.pisos.com/venta/pisos-el_carme/',
 'https://www.pisos.com/venta/pisos-el_mercat/',
 'https://www.pisos.com/venta/pisos-ciutat_vella_el_pilar/',
 'https://www.pisos.com/venta/pisos-la_seu/',
 'https://www.pisos.com/venta/pisos-la_xerea/',
 'https://www.pisos.com/venta/pisos-sant_francesc/',
 'https://www.pisos.com/ven

In [12]:
from bs4 import BeautifulSoup
import requests
import time

# Suponiendo que urls_barrios es tu lista de URLs de los barrios
diccionario_casas = {}
anuncios_por_barrio = []
i = 0
for url_barrio in urls_barrios:
    page_number = 1
    
    while page_number <= 2:  # Limita el número de páginas a 2
        url = f"{url_barrio}{page_number}"
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            anuncios = soup.find_all(class_='ad-preview')
            
            if anuncios:
                print(f"Se encontraron {len(anuncios)} anuncios en la página {url}")
                for anuncio in anuncios:
                    enlace = anuncio.find('a')['href']
                    titulo = anuncio.find(class_='ad-preview__title').text.strip()
                    subtitulo = anuncio.find(class_='p-sm ad-preview__subtitle').text.strip()
                    precio = anuncio.find(class_='ad-preview__price').text.strip()
                    
                    
                    # Agregar los detalles del anuncio a la lista
                    diccionario_casas["https://www.pisos.com"+enlace] = {f'{i}': {
                        'Precio': precio,
                        'Titulo': titulo,
                        'Subtitulo': subtitulo,
                        
                        
                    }}
                    i += 1
                    anuncios_por_barrio.append(anuncio_info)
                    
                    print(f"Enlace: {enlace}, Anuncio: {titulo}, Subtítulo: {subtitulo}, Precio: {precio}")
                    
                page_number += 1
            else:
                print(f"No se encontraron más anuncios en la página {url}, pasando a la siguiente URL...")
                break
        else:
            print(f"Error al obtener la página {url}. Código de estado: {response.status_code}")
            break
        
        # Espera un tiempo entre solicitudes para evitar ser bloqueado por el servidor
        time.sleep(2)

# Ahora anuncios_por_barrio es la lista que contiene los detalles básicos de todos los anuncios recopilados

Se encontraron 31 anuncios en la página https://www.pisos.com/venta/pisos-algiros/1
Enlace: /comprar/piso-illa_perduda46022-15930212707_994429/, Anuncio: Piso en Plaza del Pintor Anglada, Subtítulo: L'Illa Perduda (Distrito Algirós. València Capital), Precio: 173.000 €
Enlace: /comprar/piso-ciutat_jardi46021-40927169083_525823/, Anuncio: Piso en Ciutat Jardí, Subtítulo: Ciutat Jardí (Distrito Algirós. València Capital), Precio: 290.000 €
Enlace: /comprar/piso-illa_perduda46022-39248701939_106700/, Anuncio: Piso en L'illa Perduda, Subtítulo: L'Illa Perduda (Distrito Algirós. València Capital), Precio: 215.000 €
Enlace: /comprar/atico-la_bega_baixa-32501684894_516364/, Anuncio: Ático en Plaza del Poeta Vicente Gaos, Subtítulo: La Bega Baixa (Distrito Algirós. València Capital), Precio: 525.000 €
Enlace: /comprar/piso-amistat-36763279229_992527/, Anuncio: Piso en Plaza de San Felipe Neri, 11, Subtítulo: L'Amistat (Distrito Algirós. València Capital), Precio: 229.000 €
Enlace: /comprar/pis

KeyboardInterrupt: 

In [26]:
urls_barrios[0]

'https://www.pisos.com/venta/pisos-algiros/'

In [58]:
url = 'https://www.pisos.com/comprar/piso-illa_perduda46022-15930212707_994429/'

configurations = {'url': f'{url}',
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
response = requests.get(url=configurations['url'], headers={'User-Agent': configurations['user-agent']})    
soup = BeautifulSoup(response.text, 'html.parser')
precio = soup.find('div', class_='price')
precio = precio.text
print(precio)
descripcion_elem = soup.find_all('div', class_='description__content')
descripcion = [descripcion.text for descripcion in descripcion_elem]
print(descripcion)
caracteristicas_titulo = soup.find_all('div', class_='features__content')

titulos = caracteristicas_titulo[0].find_all('span', class_='features__label')
valores = caracteristicas_titulo[0].find_all('span', class_='features__value')
caracteristicas = [titulo.text for titulo in titulos]
values = [valor.text for valor in valores]
print(caracteristicas, values)
ot_titulos = caracteristicas_titulo[1].find_all('span', class_='features__label')
ot_valores = caracteristicas_titulo[1].find_all('span', class_='features__icon')
ot_caracteristicas = [titulo.text for titulo in ot_titulos]
ot_values = [valor.text for valor in ot_valores]
print(ot_caracteristicas, ot_values)



173.000 €


['\n\t\t\tVivienda junto a blasco ibañez\r87m² segun catastro. Distribuidos en tres dormitorios dobles con techos altos\rCompletamente exterior,bonito balcon una bonita vista, con salida desde el salon y dormitorio\rBaño con ducha y cocina independiente \rMuy luminoso con una orientacion norte-sur.\rSuelo de madera ,puertas de nogal,\rAire acondicionado en salon.\rIdeal para estudiantes por su cercania a facultades y a serreria\r (actualmente esta alquilado a estudiantes hasta junio)\r\n']
['Superficie construida: ', 'Superficie útil: ', 'Habitaciones: ', 'Baños: ', 'Planta: ', 'Antigüedad: ', 'Conservación: ', 'Referencia: '] ['87 m²', '87 m²', '3', '1', '5ª', 'Más de 50 años', 'A reformar', '994429-car01905']
['Amueblado'] ['']


In [1]:
from bs4 import BeautifulSoup
import requests
import time

# Suponiendo que urls_barrios es tu lista de URLs de los barrios
diccionario_casas = {}
anuncios_por_barrio = []
i = 0
for url_barrio in urls_barrios:
    page_number = 1
    
    while page_number <= 2:  # Limita el número de páginas a 2
        url = f"{url_barrio}{page_number}"
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            anuncios = soup.find_all(class_='ad-preview')
            
            if anuncios:
                print(f"Se encontraron {len(anuncios)} anuncios en la página {url}")
                for anuncio in anuncios:

                    configurations = {
                        'url': f'{anuncio}',
                        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
                    response = requests.get(url=configurations['url'], headers={'User-Agent': configurations['user-agent']})    
                    soup = BeautifulSoup(response.text, 'html.parser') 

                    descripcion_elem = soup.find('div', class_='in-readAll')
                    descripcion = None if descripcion_elem is None else descripcion_elem.text
                    caracteristicas_titulo = soup.find_all('dl', class_='in-realEstateFeatures__list')                    
                    
                    # Agregar los detalles del anuncio a la lista
                    diccionario_casas["https://www.pisos.com"+enlace] = {f'{i}': {
                        'Precio': precio,
                        'Titulo': titulo,
                        'Subtitulo': subtitulo,
                        
                        
                    }}
                    i += 1
                    anuncios_por_barrio.append(anuncio_info)
                    
                    print(f"Enlace: {enlace}, Anuncio: {titulo}, Subtítulo: {subtitulo}, Precio: {precio}")
                    
                page_number += 1
            else:
                print(f"No se encontraron más anuncios en la página {url}, pasando a la siguiente URL...")
                break
        else:
            print(f"Error al obtener la página {url}. Código de estado: {response.status_code}")
            break
        
        # Espera un tiempo entre solicitudes para evitar ser bloqueado por el servidor
        time.sleep(2)

# Ahora anuncios_por_barrio es la lista que contiene los detalles básicos de todos los anuncios recopilados

NameError: name 'anuncios_por_barrio' is not defined

In [ ]:
for link in unique_links:
    configurations = {
        'url': f'{link}',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(url=configurations['url'], headers={'User-Agent': configurations['user-agent']})    
    soup = BeautifulSoup(response.text, 'html.parser')    

    descripcion_elem = soup.find('div', class_='in-readAll')
    descripcion = None if descripcion_elem is None else descripcion_elem.text
    caracteristicas_titulo = soup.find_all('dl', class_='in-realEstateFeatures__list')
    
    if len(caracteristicas_titulo) < 4:
        
        titulos = caracteristicas_titulo[0].find_all('dt', class_='features__label')
        valores = caracteristicas_titulo[0].find_all('dd', class_='features__value')

        caracteristicas = [titulo.text for titulo in titulos]
        values = [valor.text for valor in valores]

        precio = caracteristicas_titulo[1].find('dd', class_='in-realEstateFeatures__value').text
            
        titol_ef = caracteristicas_titulo[2].find_all('dt', class_='in-realEstateFeatures__title')
        ef_v = caracteristicas_titulo[2].find_all('dd', class_='in-realEstateFeatures__value')
        tit_ef = [titulo.text for titulo in titol_ef]
        ef = [valor.text for valor in ef_v]
        
        otr = None
    
    else:
        
        otros = caracteristicas_titulo[0].find_all('div', class_='nd-badge')
        otr = [valor.text for valor in otros]

        titulos = caracteristicas_titulo[1].find_all('dt', class_='in-realEstateFeatures__title')
        valores = caracteristicas_titulo[1].find_all('dd', class_='in-realEstateFeatures__value')

        caracteristicas = [titulo.text for titulo in titulos]
        values = [valor.text for valor in valores]

        precio = caracteristicas_titulo[2].find('dd', class_='in-realEstateFeatures__value').text
            
        titol_ef = caracteristicas_titulo[3].find_all('dt', class_='in-realEstateFeatures__title')
        ef_v = caracteristicas_titulo[3].find_all('dd', class_='in-realEstateFeatures__value')
        tit_ef = [titulo.text for titulo in titol_ef]
        ef = [valor.text for valor in ef_v]

    # Crear el diccionario para almacenar la información
    diccionario_casas[link] = {
        'Precio': precio,
        'Descripcion': descripcion,
        'Otras_caractericas': otr,
        'Caracteristicas': dict(zip(caracteristicas, values)),
        'Eficiencia_energetica': dict(zip(tit_ef, ef))
    }

In [92]:
with open('anuncios_data_simple.pkl', 'wb') as f:
    pickle.dump(anuncios_por_barrio, f)

In [4]:
with open('anuncios_data_simple.pkl', 'rb') as f:
    anuncios_por_barrio = pickle.load(f)

In [5]:
anuncios_por_barrio

[{'titulo': 'Piso en Plaza del Pintor Anglada',
  'subtitulo': "L'Illa Perduda (Distrito Algirós. València Capital)",
  'precio': '173.000 €',
  'enlace': 'pisos.com/comprar/piso-illa_perduda46022-15930212707_994429/',
  'datos': '173.000 €'},
 {'titulo': 'Piso en Ciutat Jardí',
  'subtitulo': 'Ciutat Jardí (Distrito Algirós. València Capital)',
  'precio': '290.000 €',
  'enlace': 'pisos.com/comprar/piso-ciutat_jardi46021-40927169083_525823/',
  'datos': '290.000 €'},
 {'titulo': 'Ático en Albors',
  'subtitulo': "L'Amistat (Distrito Algirós. València Capital)",
  'precio': '293.500 €',
  'enlace': 'pisos.com/comprar/atico-amistat-25899796564_103800/',
  'datos': '293.500 €'},
 {'titulo': "Piso en L'illa Perduda",
  'subtitulo': "L'Illa Perduda (Distrito Algirós. València Capital)",
  'precio': '215.000 €',
  'enlace': 'pisos.com/comprar/piso-illa_perduda46022-39248701939_106700/',
  'datos': '215.000 €'},
 {'titulo': "Piso en L'Amistat",
  'subtitulo': "L'Amistat (Distrito Algirós. V